In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from FHIRClient import FHIRClient
from ml_on_fhir import MLOnFHIR
from fhir_objects.Patient import Patient

from sklearn.metrics import accuracy_score, roc_curve, auc

import logging
logging.basicConfig(level=logging.DEBUG)

In [8]:
client = FHIRClient(service_base_url='http://localhost:8080/baseDstu3')

In [9]:
pats = client.get_all_patients()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8080
DEBUG:urllib3.connectionpool:http://localhost:8080 "GET /baseDstu3/Patient?_count=100& HTTP/1.1" 200 None


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier

ml_fhir = MLOnFHIR(Patient, feature_attrs=['birthDate'], label_attrs=['gender'])
X, y, trained_clf = ml_fhir.fit(pats, DecisionTreeClassifier())

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed


In [17]:
fpr, tpr, _ = roc_curve(y, trained_clf.predict(X))
auc(fpr, tpr)

0.8989813242784381